In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

import time
from tqdm.notebook import tqdm

import datetime
from datetime import timedelta

from datetime import datetime
import re

In [143]:
data = pd.read_excel('확진자정보.xlsx')
del data['Unnamed: 0']

In [144]:
data

,감염경로,거주지,날짜,세부정보,퇴원현황,환자번호
0,해외 접촉(중국 우한시),강서구,1/24,이동경로 공개기간 경과,퇴원,#2
1,해외 접촉(중국 우한시),마포구,1/30,이동경로 공개기간 경과,퇴원,#7
2,#3 접촉,종로구,1/30,이동경로 공개기간 경과,퇴원,#6
3,해외 접촉(중국 우한시),중랑구,1/30,이동경로 공개기간 경과,퇴원,#5
4,#6 접촉,종로구,1/31,이동경로 공개기간 경과,퇴원,#11
...,...,...,...,...,...,...
21954,확인 중,타시도,9/08,확인 중,NaN,21457
21955,기타 확진자 접촉,관악구,9/08,확인 중,NaN,21556
21956,확인 중,도봉구,9/08,확인 중,NaN,21459
21957,관악구 가족모임,은평구,9/08,확인 중,NaN,21545


In [145]:
data['퇴원현황'].fillna('입원',inplace=True)
data['세부정보'].fillna('없음',inplace=True)

In [146]:
data.dropna(inplace=True)
data = data.reset_index(drop=True)

In [147]:
data.drop_duplicates(inplace=True)

In [148]:
data.reset_index(drop=True,inplace=True)

In [149]:
for i in range(len(data)):
    data['환자번호'][i] = str(data['환자번호'][i])
    data['감염경로'][i] = str(data['감염경로'][i])
    data['거주지'][i] = str(data['거주지'][i])
    data['퇴원현황'][i] = str(data['퇴원현황'][i])

In [150]:
for i in tqdm(range(len(data))):
    data['환자번호'][i] = data['환자번호'][i].replace('#','')

In [151]:
for i in tqdm(range(len(data))):
    data['감염경로'] = data['감염경로'].replace('#','')

In [152]:
data = data[data['환자번호']!='미부여']
data = data.drop_duplicates('환자번호')
data = data.reset_index(drop=True)

In [153]:
from konlpy.tag import Twitter
from collections import Counter
from konlpy.tag import Okt

okt = Okt()

In [154]:
for i in range(len(data)):
    data["감염경로"][i] = okt.pos(data['감염경로'][i])

In [155]:
total = []
for i in range(len(data)):
    snt = data['감염경로'][i]
    category = []
    word = []
    for j in range(len(snt)): 
        category.append(snt[j][1])
        word.append(snt[j][0])
    if 'Number' in category:
        total.append(snt[1][0])
    else:    
        total.append("".join(word))

In [156]:
len(data)
len(total)

19738

19738

In [157]:
data['virus_route'] = total

In [15]:
data.loc[8,'virus_route'] =  2621
data.loc[8,'virus_route'] = str(data.loc[8,'virus_route'])

In [16]:
data.loc[128,'virus_route'] = 6921
data.loc[128,'virus_route'] = str(data.loc[128,'virus_route'])

In [158]:
for i in range(len(data)):
    data['virus_route'][i] = data['virus_route'][i].rstrip('관련')
    data['virus_route'][i] = data['virus_route'][i].rstrip('접촉')

- 해외가 들어가면 해외로 분류

In [159]:
idx = data.loc[data['virus_route'].str.contains('해외')].index

for i in idx:
    data['virus_route'][i] = '해외'

- 확인중이 들어가면 확인중으로 분류

In [160]:
idx = data.loc[data['virus_route'].str.contains('확인중')].index

for i in idx:
    data['virus_route'][i] = '확인중'

- 타시도확진자가 들어가면 타시도확진자

In [161]:
idx = data.loc[data['virus_route'].str.contains('타시도확진자')].index

for i in idx:
    data['virus_route'][i] = '타시도확진자'

- 신천지가 들어가면 대구신천지

In [162]:
idx = data.loc[data['virus_route'].str.contains('신천지')].index

for i in idx:
    data['virus_route'][i] = '대구신천지'

- 명성이 들어가면 대구명성교회

In [163]:
idx = data.loc[data['virus_route'].str.contains('명성')].index

for i in idx:
    data['virus_route'][i] = '대구명성교회'

- 대구가 들어가면 대구

In [164]:
idx = data.loc[data['virus_route'].str.contains('대구')].index

for i in idx:
    data['virus_route'][i] = '대구'

- 고양시 반석교회

In [165]:
idx = data.loc[data['virus_route'].str.contains('고양시반석')].index

for i in idx:
    data['virus_route'][i] = '고양시반석교회'

- 골드 트레인

In [166]:
idx = data.loc[data['virus_route'].str.contains('골드트레인')].index

for i in idx:
    data['virus_route'][i] = '골드트레인'

- 케네디 상가

In [167]:
idx = data.loc[data['virus_route'].str.contains('케네디상가')].index

for i in idx:
    data['virus_route'][i] = '케네디상가'

- 성북구사랑제일교회가 들어가면 성북구사랑제일교회

In [168]:
idx = data.loc[data['virus_route'].str.contains('성북구사랑제일교회')].index

for i in idx:
    data['virus_route'][i] = '성북구사랑제일교회'

In [169]:
del data['감염경로']

In [46]:
sorted(data['virus_route'].unique().tolist())

['#2621번',
 '#6921접촉자',
 '10015',
 '10044',
 '10054',
 '10084',
 '10115',
 '10142',
 '10377',
 '10407',
 '11196',
 '1247',
 '136',
 '14483',
 '14488',
 '29',
 '3',
 '30',
 '5',
 '56',
 '6',
 '6727',
 '6832',
 '6937',
 '6959',
 '7280',
 '7500',
 '7573',
 '780',
 '7914',
 '7923',
 '794',
 '8044',
 '8057',
 '8063',
 '8079',
 '8138',
 '8196',
 '8245',
 '8271',
 '83',
 '8486',
 '8601',
 '8646',
 '8669',
 '8709',
 '8717',
 '8791',
 '8881',
 '9020',
 '9219',
 '9287',
 '935',
 '9364',
 '9525',
 '9572',
 '9621',
 '9641',
 '9734',
 '9765',
 '9800',
 '9912',
 '9946',
 'KT가좌지사',
 'kb생명보험',
 '가족',
 '감염경로조사중',
 '강남구CJ텔레닉스',
 '강남구CJ텔레닉스[잠언의료기]',
 '강남구K보건산업',
 '강남구개인사저경비팀',
 '강남구동인교회',
 '강남구디와이디벨로먼트',
 '강남구럭키사우나',
 '강남구사무실',
 '강남구사무실K빌딩',
 '강남구삼성화재',
 '강남구소재공기업',
 '강남구소재금융회사',
 '강남구소재대우디오빌플러스',
 '강남구소재마사지샵',
 '강남구소재병원',
 '강남구소재아파트',
 '강남구소재요양병원',
 '강남구소재콜센터',
 '강남구신도벤처타워',
 '강남구어학원관련Ⅰ',
 '강남구어학원관련Ⅱ',
 '강남구역삼동모임',
 '강남구역삼역',
 '강남구연기학원',
 '강남구음식점Ⅱ',
 '강남구이동통신종사자',
 '강남구커피전문점',
 '강남구콜센터',
 '강남구콜센터Ⅱ',
 '강

In [30]:
num = ['10015', '10044', '10054', '10084', '10115', '10142','10377', '10407', '11196', '1247',
 '136', '14483', '14488', '2621', '29', '3', '30', '5', '56', '6', '6727', '6832', '6921',
 '6937', '6959', '7280', '7500', '7573', '780', '7914', '7923', '794', '8044', '8057', '8063',
 '8079', '8138', '8196', '8245', '8271', '83', '8486', '8601', '8646', '8669', '8709',
 '8717', '8791', '8881', '9020', '9219', '9287', '935', '9364', '9525', '9572', '9621',
 '9641', '9734', '9765','9800', '9912', '9946']

- 지역 혹은 단체가 나와있는 경우에 숫자 매핑

In [31]:
a = []
aid = []
b = []
bid = []
for i in range(len(data)):
    if data['virus_route'][i] in num:
        a.append(data['virus_route'][i])
        aid.append(data['환자번호'][i])
    else:
        b.append(data['virus_route'][i])
        bid.append(data['환자번호'][i])

In [32]:
df_num = pd.DataFrame({'환자번호': aid , 'virus_route' : a}) # 확진자 id가 있는 것만 추출
df_str = pd.DataFrame({'환자번호': bid ,'virus_route' : b}) # 집단인 경우만 추출

In [33]:
df_num['label'] = pd.to_numeric(df_num['virus_route']) # 문자열숫자(확진자번호)를 숫자형

In [93]:
df_str = pd.merge(df_str,where_df,how='left')

In [94]:
df = pd.concat([df_num,df_str]) # 최종 concat (행으로붙이기)

-  500~649는 집단감염 번호이므로 실제 환자번호와 무관

In [95]:
data = pd.merge(data,df,how='left')

In [96]:
data.rename(columns={'virus_route' : '발생지',
                     'label':'감염자'},inplace=True)

- 날짜 전처리

In [170]:
for i in tqdm(range(len(data))):
    if type(data["날짜"][i]) == str:
        data["날짜"][i] = "2020-"+data["날짜"][i].replace("/","-")

In [171]:
idx = data[data['날짜'].str.contains('21.01')].index

In [172]:
data.dtypes

거주지            object
날짜             object
세부정보           object
퇴원현황           object
환자번호           object
virus_route    object
dtype: object

In [173]:
data = data.drop(idx).reset_index(drop=True)
data

,거주지,날짜,세부정보,퇴원현황,환자번호,virus_route
0,강서구,2020-1-24,이동경로 공개기간 경과,퇴원,2,해외
1,마포구,2020-1-30,이동경로 공개기간 경과,퇴원,7,해외
2,종로구,2020-1-30,이동경로 공개기간 경과,퇴원,6,3
3,중랑구,2020-1-30,이동경로 공개기간 경과,퇴원,5,해외
4,종로구,2020-1-31,이동경로 공개기간 경과,퇴원,11,6
...,...,...,...,...,...,...
18959,영등포구,2020-9-08,확인 중,입원,21522,타시도확진자
18960,관악구,2020-9-08,확인 중,입원,21554,기타확진자
18961,관악구,2020-9-08,확인 중,입원,21546,기타확진자
18962,타시도,2020-9-08,확인 중,입원,21457,확인중


In [174]:
# 12월
idx = data[data['날짜'].str.contains('20.12')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

In [177]:
pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][8:10]+"-"+a[i][11:13])

data.loc[idx,'날짜'] = pre

In [179]:
# 11월
idx = data[data['날짜'].str.contains('20.11')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][8:10]+"-"+a[i][11:13])

data.loc[idx,'날짜'] = pre

In [181]:
data.loc[idx,'날짜']

821     2020-11-01
822     2020-11-01
823     2020-11-01
824     2020-11-01
825     2020-11-01
           ...    
3716    2020-11-30
3717    2020-11-30
3718    2020-11-30
3719    2020-11-30
3720    2020-11-30
Name: 날짜, Length: 2900, dtype: object

In [198]:
# 8월
idx = data[data['날짜'].str.contains('2020-8')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][5]+"-"+a[i][7:9])

data.loc[idx,'날짜'] = pre

In [214]:
data[data['날짜'].str.contains('20.11')]

,거주지,날짜,세부정보,퇴원현황,환자번호,virus_route
821,마포구,2020-11-01,타시도 확진자 접촉,퇴원,26644,타시도확진자
822,기타,2020-11-01,해외 유입,퇴원,26652,해외
823,동대문구,2020-11-01,동대문구 에이스희망케어센터 관련,퇴원,26658,동대문구에이스희망케어센터
824,도봉구,2020-11-01,동대문구 에이스희망케어센터 관련,퇴원,26659,동대문구에이스희망케어센터
825,강서구,2020-11-01,기타 확진자 접촉,퇴원,26691,기타확진자
...,...,...,...,...,...,...
3716,타시도,2020-11-30,강서구 댄스교습관련 시설,퇴원,34473,강서구댄스교습관련시설
3717,강남구,2020-11-30,기타 확진자 접촉,퇴원,34534,기타확진자
3718,중구,2020-11-30,감염경로 조사중,퇴원,34525,감염경로조사중
3719,영등포구,2020-11-30,강서구 소재 댄스교습시설 연관 병원,퇴원,34472,강서구소재댄스교습시설연관병원


In [216]:
data[data['날짜'].str.contains('20.08')]

,거주지,날짜,세부정보,퇴원현황,환자번호,virus_route
820,영등포구,2020-20.08.25.,감염경로 조사중,사망,28770,감염경로조사중


In [217]:
# 08월
idx = data[data['날짜'].str.contains('20.08')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][8:10]+"-"+a[i][11:13])

data.loc[idx,'날짜'] = pre

In [200]:
# 9월
idx = data[data['날짜'].str.contains('2020-9')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][5]+"-"+a[i][7:9])

data.loc[idx,'날짜'] = pre

In [203]:
# 10월
idx = data[data['날짜'].str.contains('2020-10')].index

a = data.iloc[idx]['날짜']
a.reset_index(drop=True,inplace=True)

pre = []
for i in range(len(a)):
    pre.append("2020-"+a[i][5:7]+"-"+a[i][8:10])

data.loc[idx,'날짜'] = pre

In [218]:
data['날짜'] = pd.to_datetime(data['날짜'])

In [229]:
data = data.sort_values('날짜').reset_index(drop=True)

- Plot에서 x축 조절을 위해 그림용 번호 만들기

In [99]:
unique = data['감염자'].unique().tolist()
len(unique)

213

In [101]:
unique = data['감염자'].unique().tolist()
df = pd.DataFrame({'감염자':unique,'그림번호': list(range(1,214))})
data = pd.merge(data,df,how='left')

In [104]:
del data['감염경로']

In [228]:
data.to_excel('network.xlsx')

In [231]:
data

,거주지,날짜,세부정보,퇴원현황,환자번호,virus_route
0,강서구,2020-01-24,이동경로 공개기간 경과,퇴원,2,해외
1,마포구,2020-01-30,이동경로 공개기간 경과,퇴원,7,해외
2,종로구,2020-01-30,이동경로 공개기간 경과,퇴원,6,3
3,중랑구,2020-01-30,이동경로 공개기간 경과,퇴원,5,해외
4,종로구,2020-01-31,이동경로 공개기간 경과,퇴원,11,6
...,...,...,...,...,...,...
18959,동작구,2020-12-31,감염경로 조사중,입원,61275,감염경로조사중
18960,영등포구,2020-12-31,감염경로 조사중,입원,61254,감염경로조사중
18961,영등포구,2020-12-31,감염경로 조사중,입원,61222,감염경로조사중
18962,강북구,2020-12-31,관악구 소재 종교시설 관련,입원,60983,관악구소재종교시설
